In [1]:
import netgen.gui
from ngsolve import *
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import inv
from scipy.sparse.linalg import spsolve
from scipy import random

In [2]:
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
Draw (mesh)


In [3]:
fes = H1(mesh, order=3,dirichlet=".*")
gfu_a = GridFunction(fes,name = "u_a")
gfu_b = GridFunction(fes,name = "u_b")
print(len(gfu_a.vec))

4342


In [7]:
def SolveCG(A,C, u_grid,f_grid,maxsteps, tol, printrate):
    r = u_grid.CreateVector()
    d = u_grid.CreateVector()
    p = u_grid.CreateVector()
    dTp = u_grid.CreateVector()
    Ap = u_grid.CreateVector()
    dC = u_grid.CreateVector()
    pTAp = u_grid.CreateVector()
    tempN = u_grid.CreateVector()
    tempd_res = u_grid.CreateVector()
    #total_err = u_grid.CreateVector()
    r.FV().NumPy()[:] = random.rand(fes.ndof)
    u_grid.data = Projector(fes.FreeDofs(), True) * r
    
    d.data = f_grid - A*u_grid
    p.data = C*d.data
    
    for k in range(maxsteps):
        Ap.data = A * p.data
        dTp = InnerProduct(d.data, p.data)

        pTAp = InnerProduct(p.data, Ap)
        alpha = dTp/pTAp

        u_grid.data = u_grid.data + alpha*p.data
        
        tempd_res = d.data
        d.data = d.data - alpha*Ap.data
        total_err = InnerProduct(tempd_res,d.data)*InnerProduct(tempd_res,d.data)
        if printrate:
            print("step:",k,"tot: ",total_err)
        #print(total_err)
        dC.data = C*d.data
        #tempN = dC*Ap
        tempN = InnerProduct(dC, Ap)
        beta = -tempN/pTAp

        p.data = C*d.data + beta*p.data
    #gfu_new = u_grid  
    return u_grid  

In [8]:
ua = fes.TrialFunction()
va = fes.TestFunction()

Aa = BilinearForm(fes)
Fa = LinearForm(fes)

Aa += ua*va*dx

Fa += va*dx

Aa.Assemble()
Fa.Assemble()


c = Preconditioner(Aa,"local",test = True)
c.Update()

gfu_new = SolveCG(Aa.mat,c.mat,gfu_a.vec,Fa.vec,200,int(1e-8),True)

gif_a = GridFunction(fes,name = "gif_a")
gif_a.vec.FV().NumPy()[:] = gfu_new
#gfu_a.vec.data = Aa.mat.Inverse(fes.FreeDofs()) * Fa.vec

Draw(gif_a)

step: 0 tot:  3.71490583837629e-10
step: 1 tot:  6.577554427412443e-11
step: 2 tot:  8.706915371029514e-12
step: 3 tot:  2.2515909151983313e-11
step: 4 tot:  4.163171213747449e-12
step: 5 tot:  1.350914466917533e-11
step: 6 tot:  3.329440019080353e-12
step: 7 tot:  9.290250684499818e-12
step: 8 tot:  3.010800511898279e-12
step: 9 tot:  6.8678575465545015e-12
step: 10 tot:  2.8289416766266454e-12
step: 11 tot:  5.3803879364383014e-12
step: 12 tot:  2.703183315948005e-12
step: 13 tot:  4.4295705940894395e-12
step: 14 tot:  2.6084998764039686e-12
step: 15 tot:  3.8022883488761106e-12
step: 16 tot:  2.5342810310259802e-12
step: 17 tot:  3.377131956480899e-12
step: 18 tot:  2.4747918602604708e-12
step: 19 tot:  3.0820711616697917e-12
step: 20 tot:  2.426407802985939e-12
step: 21 tot:  2.872932400086487e-12
step: 22 tot:  2.3866247149379308e-12
step: 23 tot:  2.721829144815606e-12
step: 24 tot:  2.3536245356274384e-12
step: 25 tot:  2.6107017011145236e-12
step: 26 tot:  2.326047768160757e-12

In [10]:
ub = fes.TrialFunction()
vb = fes.TestFunction()

Ab = BilinearForm(fes)
Fb = LinearForm(fes)

Ab += grad(ua)*grad(va)*dx

Fb += va*dx

Ab.Assemble()
Fb.Assemble()


c = Preconditioner(Ab,"local",test = True)
c.Update()

gfu_new = SolveCG(Ab.mat,c.mat,gfu_b.vec,Fb.vec,2000,int(1e-8),True)

gif_b = GridFunction(fes,name = "gif_b")
gif_b.vec.FV().NumPy()[:] = gfu_new
#gfu_a.vec.data = Aa.mat.Inverse(fes.FreeDofs()) * Fa.vec

Draw(gif_b)

step: 0 tot:  1710.6817974009473
step: 1 tot:  945.9333958908745
step: 2 tot:  176.04171044035067
step: 3 tot:  160.1852730824533
step: 4 tot:  60.13970981435543
step: 5 tot:  60.138954805795876
step: 6 tot:  29.83568862060298
step: 7 tot:  30.189768127764317
step: 8 tot:  17.58500005602577
step: 9 tot:  17.651888718441562
step: 10 tot:  11.438188306190469
step: 11 tot:  11.36972010193627
step: 12 tot:  7.938209059129334
step: 13 tot:  7.833206203738348
step: 14 tot:  5.771167642903472
step: 15 tot:  5.669752416186608
step: 16 tot:  4.345763168005537
step: 17 tot:  4.260449877660385
step: 18 tot:  3.363714392679264
step: 19 tot:  3.296003352484247
step: 20 tot:  2.6617548973433514
step: 21 tot:  2.609396365695023
step: 22 tot:  2.144831252987958
step: 23 tot:  2.1047801701746858
step: 24 tot:  1.7547717792043929
step: 25 tot:  1.7242508497285232
step: 26 tot:  1.4544295879522893
step: 27 tot:  1.4312013081253057
step: 28 tot:  1.2191953587743565
step: 29 tot:  1.2015430273271033
step: 